### In order for Python to find the Spark, download the findspark library and start it with findspark.init() function.

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=6350dd3766bf0c81b5c8cb90872011eb93e2987faf5c7ea02c364b268863bb4f
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [ ]:
import findspark
findspark.init()

### In order to work with RDDs, we need to create a SparkContext.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf


## Since we write local [*] in the master, it will use all cores in our machine. If we said local [4] it will work with 4 cores.

## getOrCreate is used to create a SparkSession if not present.

In [ ]:
spark=SparkSession.builder\
    .master("local[*]")\
    .appName("WordCount")\
    .getOrCreate()

In [ ]:
sc=spark.sparkContext

## Read Data - RomeoJuliet Txt File

In [ ]:
veri_dosyasi="romeojuliet.txt"

In [ ]:
shakespeare_rdd=sc.textFile(veri_dosyasi)

In [ ]:
shakespeare_rdd.take(100)

['',
 "                    WILLIAM SHAKESPEARE'S",
 '',
 '                       ROMEO & JULIET',
 '',
 '   ADAPTED FOR THE SCREEN BY CRAIG PEARCE AND BAZ LUHRMANN',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '                                       FINAL SHOOTING SCRIPT',
 '',
 '                                             October 6, 1995',
 '',
 '',
 '',
 '',
 'EXT.  HIGHWAY.  AFTERNOON.',
 '',
 'A ribbon of freeway stretching into a blue and pink late',
 'afternoon sky. A huge dark sedan, windows tinted gold,',
 'headlights blazing, powers directly for us.',
 '',
 'CUT TO: A heavy, low-slung, pickup truck traveling toward',
 'the sedan.',
 '',
 'WIDE SHOT: Sky, freeway, the cars closing.',
 '',
 'TIGHT ON: The sedan.',
 '',
 'TIGHT ON: The pickup.',
 '',
 'Like thunderous, jousting opponents, the cars pass in a',
 'deafening cacophony of noise.',
 '',
 'INT.  TRUCK.  AFTERNOON.',
 '',
 'TIGHT ON: 

In [ ]:
shakespeare_rdd.count()

6247

# Remove Punctuation and Transform All Words to Lowercase.

### To exclude punctuation values and convert all words to lowercase, we wrote a function like the one below.

In [ ]:
def lower_clean_str(x):
  punc='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
  lowercased_str = x.lower()
  for ch in punc:
    lowercased_str = lowercased_str.replace(ch, '')
  return lowercased_str

In [ ]:
shakespeare_rdd = shakespeare_rdd.map(lower_clean_str)

In [ ]:
shakespeare_rdd.take(40)

['',
 '                    william shakespeares',
 '',
 '                       romeo  juliet',
 '',
 '   adapted for the screen by craig pearce and baz luhrmann',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '                                       final shooting script',
 '',
 '                                             october 6 1995',
 '',
 '',
 '',
 '',
 'ext  highway  afternoon']

## We use split function to separate the words in all lines .

In [ ]:
shakespeare_rdd=shakespeare_rdd.flatMap(lambda satir: satir.split(" "))

In [ ]:
shakespeare_rdd.take(5)

['', '', '', '', '']

## We do a filtering below to exclude whitespaces.

In [ ]:
shakespeare_rdd = shakespeare_rdd.filter(lambda x:x!='')

In [ ]:
shakespeare_rdd.take(4)

['william', 'shakespeares', 'romeo', 'juliet']

## Count how many times each word occurs.
### To make this calculation we can apply the “reduceByKey” transformation on (key,val) pair RDD. We need to first convert “shakespeare_rdd” to (key,val) pair RDD.

### In this new (key,val) pair RDD (shakespeare_count), key is the word and val is 1 for each word in RDD (1 represents the number for the each word in “shakespeare_rdd”).


In [ ]:
shakespeare_count=shakespeare_rdd.map(lambda  word:(word,1))

In [ ]:
shakespeare_count.take(4)

[('william', 1), ('shakespeares', 1), ('romeo', 1), ('juliet', 1)]

## Apply ReduceByKey to find frequent words

In [ ]:
shakespeare_count_RBK=shakespeare_count.reduceByKey(lambda x,y:(x+y)).sortByKey()

In [ ]:
shakespeare_count_RBK.take(40)

[('1995', 1),
 ('21', 1),
 ('6', 1),
 ('60', 2),
 ('9mm', 2),
 ('a', 563),
 ('abandoned', 1),
 ('able', 1),
 ('about', 3),
 ('above', 12),
 ('abra', 24),
 ('abras', 3),
 ('abroad', 1),
 ('abrupt', 1),
 ('abruptly', 5),
 ('absolved', 1),
 ('abuse', 2),
 ('abuses', 1),
 ('accidentally', 1),
 ('accompanied', 1),
 ('according', 1),
 ('accusation', 1),
 ('accustomed', 2),
 ('ache', 1),
 ('aches', 1),
 ('achingly', 2),
 ('acoustic', 1),
 ('across', 24),
 ('actually', 1),
 ('adagio', 1),
 ('adapted', 1),
 ('address', 1),
 ('addressed', 1),
 ('addresses', 1),
 ('adept', 1),
 ('adieu', 4),
 ('adjacent', 1),
 ('adjoining', 1),
 ('adjust', 1),
 ('admired', 1)]

### We want to sort the most frequent words in descending order. As the first step, we switch (key,val) pairs as (val,key).

In [ ]:
shakespeare_count_RBK=shakespeare_count_RBK.map(lambda x:(x[1],x[0]))

In [ ]:
shakespeare_count_RBK.take(5)

[(1, '1995'), (1, '21'), (1, '6'), (2, '60'), (2, '9mm')]

## We see that the most common word is "the". However, these values are words that we call stopwords which brings value to our analysis.

In [ ]:
shakespeare_count_RBK.sortByKey(False).take(10)

[(1372, 'the'),
 (563, 'a'),
 (506, 'to'),
 (469, 'of'),
 (464, 'romeo'),
 (461, 'and'),
 (258, 'in'),
 (251, 'juliet'),
 (246, 'is'),
 (224, 'i')]

## To exclude stopwords words, we download the nltk library and get the list of English stopwords.

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
stopwords =stopwords.words('english')
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

## When we exclude stopwords values, we see that the word "romeo" is the most common.

In [ ]:
shakespeare_count_RBK = shakespeare_count_RBK.filter(lambda x: x[1] not in stopwords).sortByKey(False)

In [ ]:
shakespeare_count_RBK.sortByKey(False).take(20)

[(464, 'romeo'),
 (251, 'juliet'),
 (143, 'mercutio'),
 (133, 'capulet'),
 (114, 'thou'),
 (111, 'benvolio'),
 (111, 'night'),
 (98, 'father'),
 (97, 'ext'),
 (96, 'close'),
 (96, 'nurse'),
 (92, 'cont'),
 (88, 'int'),
 (87, 'cut'),
 (84, 'car'),
 (82, 'love'),
 (81, 'laurence'),
 (79, 'tybalt'),
 (71, 'gloria'),
 (66, 'day')]